In [10]:
import imp
from click import prompt
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt = load_prompt('./prompt.yaml')


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts,input_variables=['character','example_question','example_answer','question'])

chain = full_prompt | chat
chain.invoke({
"character":'Pirate',
"example_question":"What is your location?",
"example_answer":"Arrg! That is a secret!! Arg Arg!",
"question":"What is your fav food?"
}
)

Arrg! Me favorite food be a good ol' plate o' fish 'n' chips! The salty taste o' the sea be me delight! Arg Arg!

AIMessageChunk(content="Arrg! Me favorite food be a good ol' plate o' fish 'n' chips! The salty taste o' the sea be me delight! Arg Arg!")